# Problem

You are playing a game vs A. A can only randomly choose any integer from 0 to 100. You can choose any number in the range. The person who chooses the highest number loses, and pays the other the number they (the winner) chose. What is your strategy? What if a second player, who can also choose like you, joins the game? In the 3 player game, the loser pays each of the winners the number they (the winner) chose.

**The objective is:**
1. Win the game
2. Be as close as possible of the loser

To achieve those objectives we have to think the **stats involved**, and then generate the **algorithm**. 

We will start with very naive approaches to studying the problem and then we will make the solution more complex.

# Explanation

_Player A_: Chooses numbers randomly.

_Player B_: Chooses numbers based on a defined strategy.

To explore the problem, we will solve it using three different approaches. The goal is to showcase diverse ways to think about, solve, and present the results while encouraging critical thinking. Among the three, **Strategy C** is the optimal one and should be used. Each strategy is explained before its corresponding code block.

**Strategy A**: Fixed Number

Player B always selects the same number across all iterations.

* Test different fixed numbers between 1 and 100.
* Analyze:
    * How many times Player A wins versus Player B.
    * The cumulative amount of money won or lost by Player B.

**Strategy B**: $J+1$ Approach

In this strategy, Player B adjusts its choice dynamically:

* Based on the number Player A selected in iteration $J$, Player B computes probabilities of the next number being higher or lower.
* Player B uses these probabilities to select its number in iteration $J+1$.

**Strategy C**: Maximizing Expected Value (MEV)

Player B chooses its number to maximize the expected value of its outcomes.

This involves calculating the potential payoffs for each number and selecting the one with the highest expected value.

In all cases, the residuals are computed like $\Delta = i_A - i_B$, so when 
* $\Delta < 0$ Player B wins, and 
* $\Delta > 0$ Player B lose.

In [1]:
# Import packages
import numpy as np
import matplotlib.pyplot as plt

from plotly.subplots import make_subplots
import plotly.graph_objects as go

from scipy.optimize import minimize_scalar


**Player A**: Random number between $0$ and $100$.

In [2]:
#
# Number generator for player A
#
def number_A(num_cases):
    # Generate a random number between 0 and 100
    return np.random.randint(0,100, num_cases, dtype=np.int32)
print(" Player A Chose: ", number_A(1))

 Player A Chose:  [77]


### **Strategy A**: User allways choose a fixed number

## Explanation

Player B plays with the same number in each iteration.

We explore resultschanging the fixed value of Player B (`cases` list).

In [3]:
# Params
number_cases=10000

# Fixed values for Player B
cases = [3,5,15,32,47,62,74,83,96]

# Figure and iterations
# Share x-axis

fig = make_subplots(rows=len(cases), 
                    cols=2, 
                    subplot_titles=("Residuals of A-B", "Histogram of residuals"),
                    shared_xaxes=True,
                    vertical_spacing=0.01)
i = 0
# Loop over all values of B
for i_B in cases:
    # Compute payout or residuals
    residuals = number_A(number_cases) - i_B
    
    # Step to plot residuals
    step=int(30)
    
    # Plot residuals
    fig.add_trace(go.Scatter(x=np.linspace(0,1000,int(len(residuals)/step)+1), 
                             y=residuals[::step], 
                             mode='lines+markers', 
                             name='Residuals'), 
                  row=i+1, col=1)
    # Set ylabel
    fig.update_yaxes(title_text="Residuals: randA - userB", row=i+1, col=1)
    
    # histogram with counts in Y-axis
    # compute historgram and media and median of residuals
    hist, bin_edges = np.histogram(residuals, bins=20)
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
    
    # Calculate mean and median
    mean_residuals = np.mean(residuals)
    median_residuals = np.median(residuals)
    if mean_residuals < 0:
        color = 'red'
    else:
        color = 'green'
    
    # Plot histogram
    fig.add_trace(go.Bar(x=bin_centers, y=hist, name='Histogram'), row=i+1, col=2)
    # Plot mean and median vlines
    fig.add_trace(go.Scatter(x=[mean_residuals, mean_residuals], 
                             y=[0, np.max(hist)], 
                             mode='lines', 
                             name='Mean', 
                             line=dict(color='red')), 
                  row=i+1, col=2)
    # Median
    fig.add_trace(go.Scatter(x=[median_residuals, median_residuals], 
                             y=[0, np.max(hist)], 
                             mode='lines', 
                             name='Median', 
                             line=dict(color='blue')), 
                  row=i+1, col=2)
    # Add annotation with mean and median
    fig.add_annotation(x=mean_residuals, y=np.max(hist)-80, 
                       text="Mean: {:.2f}".format(mean_residuals), 
                       showarrow=True, 
                       arrowhead=1, 
                       yshift=10,
                       font=dict(color=color), 
                       row=i+1, col=2)
    fig.add_annotation(x=median_residuals, y=np.max(hist),
                          text="Median: {:.2f}".format(median_residuals), 
                          showarrow=True, 
                          arrowhead=1, 
                          yshift=10,
                          font=dict(color=color), 
                          row=i+1, col=2)
    i=i+1
fig.update_layout(
    autosize=False,
    width=1200,
    height=1600,
)
fig.show()   

**Conclusion** No conclusions. Naive approach. Just to study the parameters and the behavior. 

### **Strategy B** Given `J` infer `J+1`

The probability distribution of chosing $i_A$ is uniform,so we can create a probabilistic model to infer the next number given the previous one. The seed in this case is 50 so equal probability for first try.

**Idea:**

Given a random number $i_A$ we compute the probability of be greater or lower value in the following iteration. $$P_{higher} = \frac{100-i_A}{100},$$ $$P_{lower} = \frac{i_A}{100}.$$

* Seed $i_0$ 
* Iteration 1: $i_B = i0$
    * Compute $i_A$
    * Compute $\Delta$
    * Update $i_0$: 
        * If $P_{lower} > P_{higher}$ --> $i_0 = \textrm{randint}(0, i_A / 2)$   
        * If $P_{higher} > P_{lower}$ --> $i_0 = \textrm{randint}(i_A, i_A + \frac{100-i_A}{100})$

In [4]:
number_cases=10000
cases_to_test = 5

fig = make_subplots(rows=len(cases), cols=2, 
                    subplot_titles=("Residuals of A-B", "Histogram of residuals"),
                    shared_xaxes=True,
                    vertical_spacing=0.01)

i = 0
for jiter in range(cases_to_test):
    # Compute payout or residuals
    residuals = []
    i0 = 50
    for _ in range(number_cases):
        prob_higher = (100-i0)/100
        prob_lower = i0/100
        i_A = number_A(1)[0]
        i_B = i0
        residuals.append(i_A - i_B)

        if prob_higher >= prob_lower:
            if i_A >= 99:
                i0 = np.random.randint(0,int(i_A/2),1)[0]
            else:
                i0 = np.random.randint(i_A,int(i_A+(100-i_A)/2),1)[0]
        elif prob_higher < prob_lower:
            if i_A <=1 :
                i0 = np.random.randint(0,100,1)[0]
            else:
                i0 = np.random.randint(0,i_A/2,1)[0]
    
    step=int(30)
    
    # Plot residuals with x-labels
    fig.add_trace(go.Scatter(x=np.linspace(0,1000,int(len(residuals)/step)+1), 
                             y=residuals[::step], 
                             mode='lines+markers', 
                             name='Residuals'), 
                  row=i+1, col=1)
    
    # histogram with counts in Y-axis
    # compute historgram and media and median of residuals
    hist, bin_edges = np.histogram(residuals, bins=20)
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
    # Calculate mean and median
    mean_residuals = np.mean(residuals)
    median_residuals = np.median(residuals)
    if mean_residuals < 0:
        color = 'red'
    else:
        color = 'green'
        
    # Plot histogram
    fig.add_trace(go.Bar(x=bin_centers, y=hist, name='Histogram'), row=i+1, col=2)
    # Plot mean and median vlines
    fig.add_trace(go.Scatter(x=[mean_residuals, mean_residuals], 
                             y=[0, np.max(hist)], 
                             mode='lines', 
                             name='Mean', 
                             line=dict(color='red')), 
                  row=i+1, col=2)
    # Median
    fig.add_trace(go.Scatter(x=[median_residuals, median_residuals],
                                y=[0, np.max(hist)], 
                                mode='lines', 
                                name='Median', 
                                line=dict(color='blue')), 
                    row=i+1, col=2)
    # Add annotation mean and median
    fig.add_annotation(x=mean_residuals, y=np.max(hist)-400, 
                       text="Mean: {:.2f}".format(mean_residuals), 
                       showarrow=True, 
                       arrowhead=1, 
                       yshift=10,
                       font=dict(color=color), 
                       row=i+1, col=2)
    fig.add_annotation(x=median_residuals, y=np.max(hist),
                            text="Median: {:.2f}".format(median_residuals), 
                            showarrow=True, 
                            arrowhead=1, 
                            yshift=10,
                            font=dict(color=color), 
                            row=i+1, col=2)
    i=i+1

fig.update_layout(
    autosize=False,
    width=1200,
    height=1200,
)
fig.show()

**Conclusion** Improve the distribution, more centered arround zero. Most of the time we lose. 

### **Strategy C** Maximize Expected Value of Wining - Losses

What is the expected value for Player B? 

* The probability of Player B wins is  $P(i_A>i_B)= \frac{100-i_B}{100} \cdot i_B$. Given $i_B$, the probability of win so $P(i_A>i_B)=\frac{100-i_B}{100}$
* The probability of Player B loses is $P(i_A<i_B) = \frac{i_B}{100} \cdot \frac {i_B}{2}$. Because we have uniform distribution the probability of chose a loser value is $\frac{i_B}{2}$ (mean) and multiplied by the probability of $i_B$ be betwen 0 and $i_B$.

So we have to maximize: $$EV(i_B) = \frac{100-i_B}{100}\cdot i_B - \frac{i_B}{100}\cdot \frac{i_B}{2} =  i_B \cdot \bigg(1 - \frac{3 i_B}{200} \bigg)$$ 

But also we prefer to be as close to $i_A$ as possible so we add a penalty term given by: $$\lambda \cdot E(|i_A - i_B|)$$

So adding the two terms reult in optimize the objective function: $$\textrm{objective}(i_B) = \textrm{expected\_value} + \lambda \cdot \textrm{penalization}(i_B).$$

In [5]:
def expected_value(ib):
    ib = np.clip(ib, 0, 100)
    fact0 = ib
    fact1 = 1 - 3*ib/200
    EV = fact0*fact1
    return EV

# Define the penalty function
# I want to penalize larger differences between i_B and i_A. It can be traeted as expected absolute difference
# with i_A having uniform distribution. There are two cases:
# 1. i_A > i_B: so i_A - i_B > 0.
#   Expected value is: E[i_A-i_B| i_A > i_B] = 1/2 * (100 - i_B)
#   Because: P(i_A>i_B)= (100-i_B)/100 and E[i_A|i_A>i_B] = (i_B+100) / 2 # Mean value of uniform distribution in range (i_B, 100)
# 2. i_A < i_B: so i_A - i_B < 0.
#   Expected value is: E[i_A-i_B| i_A < i_B] = 1/2 * i_B
#   Because: P(i_A<i_B)= i_B/100 and E[i_A|i_A<i_B] = i_B / 2 # Mean value of uniform distribution in range (0, i_B)
#
#   So, the expected value of the penalty is:
# Total penalty:
#   Win: (100 - i_B) / 100 * (100 - i_B) / 2 
#   Loss: i_B / 100 * i_B / 2
def penalty(i_B):
    i_B = np.clip(i_B, 0, 100)  # Ensure i_B is within valid range
    penalty_win = (100-i_B)**2/200  # Winning penalty
    penalty_loss = (i_B ** 2) / 200   # Losing penalty
    return penalty_win + penalty_loss

# Define the combined objective function
def objective(i_B, lambda_weight=0.):
    ev = expected_value(i_B)
    pen = penalty(i_B)
    return -(ev - lambda_weight * pen)  # Negate for minimization

# Low Lambda: maximize expected value
# High Lambda: minimize penalty
result = minimize_scalar(objective, bounds=(0, 100), method='bounded')
optimal_b = result.x
print("Optimal B: ", optimal_b)


Optimal B:  33.333333333333336


It is missing optimize $\lambda$ value.

In [6]:
ib_values = np.linspace(0,100, num=1000)

# Create plot in plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=ib_values, y=expected_value(ib_values), name='Expected Value'))
fig.add_trace(go.Scatter(x=ib_values, y=penalty(ib_values), name='Penalty'))
fig.add_trace(go.Scatter(x=[optimal_b], y=[expected_value(optimal_b)], mode='markers', name='Optimal B'))
fig.update_layout(title='Objective Function Components', xaxis_title='i_B', yaxis_title='Value')
fig.show()

#plt.plot(ib_values, objective(ib_values, lambda_weight=1), label='Lambda = 1')
#plt.plot(ib_values, objective(ib_values, lambda_weight=0.5), label='Lambda = 0.5')
#plt.plot(ib_values, objective(ib_values, lambda_weight=5), label='Lambda = 5')

### **3 players scenario**

The winning condition changes:
* My number must to be lower than A or C players
* I have to add new term to penalize big difference betwen $i_B$ and $i_C$



